# Day 5 Session 2 – Joint Control and Inverse Kinematics - Part 1

Now that we have a robot in our simulator, let’s learn how to **move** it. This session covers controlling robot joints (as if they had motors) and introduces the concept of **inverse kinematics (IK)** – figuring out the joint angles needed to get a robot’s end-effector (e.g., hand) to a desired position.


## Joints and Motors: How Robots Move

Physical robots have motors or actuators at their joints. In simulation, we control joints by specifying angles (for position control), velocities, or forces/torques. PyBullet provides a convenient function `setJointMotorControl2` to command a joint.

We can control a joint in various modes:
- **Position control**: we tell the joint to move to a target angle. The simulator will move it as if a motor is trying to reach that angle.
- **Velocity control**: we set a target speed for the joint to rotate.
- **Torque control**: we directly apply a force/torque on the joint (advanced mode, not needed for now).

For our simple two-link robot, position control is easiest to understand – we set a desired angle for each joint.

**Important**: If we want the simulation to actually move the joint over time, we need to call stepSimulation() repeatedly. If we just set a new joint target and instantly read the state, the simulation might not have progressed. We can also enable real-time simulation, but here we’ll step manually to keep control.


## Inverse Kinematics (IK) Concept

Normally, if we want a robot arm to put its hand at a certain spot, we have to figure out what joint angles will achieve that. **Inverse kinematics** is the mathematical process of calculating the joint parameters needed to place the end of a kinematic chain (like the robot’s hand) at a desired position and orientation. In simpler terms, forward kinematics answers “Where is the hand if the joints are at angles θ1, θ2, ...?” while inverse kinematics answers “What joint angles θ1, θ2, ... will put the hand at position (X, Y, Z)?”.

IK can be tricky because:
- There might be multiple possible solutions (e.g., an arm can sometimes reach a point in different ways).
- Or no solution if the target is out of reach.
- It often involves solving several equations (often using geometry or linear algebra).

PyBullet has a built-in IK solver (`p.calculateInverseKinematics`) which simplifies this for us. We give it the target position (and optionally orientation) for the end-effector, and it returns a list of joint angles that (approximately) achieve that.

In this session, we’ll:
- Manually command joint movements.
- Use PyBullet’s IK to move the end-effector to a target.

Let’s get started by reconnecting to our simulation and getting our robot ready.

**Code**: Setup simulation and load the robot (fresh start).


In [ ]:
!pip install -q pybullet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 7.9 MB/s eta 0:00:00


In [ ]:
# Connect to PyBullet and load the simple two-link robot
import pybullet as p
import pybullet_data
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0, 0, -10)
p.loadURDF("plane.urdf")
# Load our simple arm (make sure the URDF from Day 4 is available)
robot_id = p.loadURDF("simple_arm.urdf", basePosition=[0,0,0], useFixedBase=True)
num_joints = p.getNumJoints(robot_id)
print("Robot loaded with ID", robot_id, "- number of joints:", num_joints)


Robot loaded with ID 1 - number of joints: 2


We now have the robot in its default pose (both joint angles 0). The end-effector of our robot (let’s define that as the tip of link2) is, in this default pose, hanging straight down from the base.

To identify the end-effector in PyBullet, we need the link index. Joint indices 0 and 1 correspond to link1 and link2 respectively (since base_link is index -1 internally). So link2 (the second link) has index 1 in PyBullet’s system. We will use that for IK calculations.


## Controlling Joints to Specific Angles

First, let’s try to manually set joint angles. We can use p.setJointMotorControl2 in position control mode.

The function needs:
- `bodyUniqueId` (our robot_id),
- `jointIndex`,
- `controlMode` (we’ll use `p.POSITION_CONTROL`),
- `targetPosition` (the desired angle in radians),
- optionally, `force` (max force the motor can apply, default might be small, so we may set a value).

Let’s move joint1 and joint2 to some non-zero angles and see what happens.

**Code**: Command joints to move to new angles.

In [ ]:
# Define target angles for the joints
target_angle_joint1 = 1.0  # radians (~57 degrees)
target_angle_joint2 = -0.5  # radians (~ -28.6 degrees)

# Send position control command for each joint
p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=target_angle_joint1, force=5*10**1)
p.setJointMotorControl2(robot_id, 1, p.POSITION_CONTROL, targetPosition=target_angle_joint2, force=5*10**1)

# Step the simulation forward to allow the joints to move
for _ in range(200):
    p.stepSimulation()

# Read back the joint angles
angle1 = p.getJointState(robot_id, 0)[0]
angle2 = p.getJointState(robot_id, 1)[0]
print(f"Joint1 angle after moving = {angle1:.2f} rad, Joint2 angle = {angle2:.2f} rad")


Joint1 angle after moving = 1.00 rad, Joint2 angle = -0.50 rad


In the above code:
- We set joint1 to about 1.0 rad (which is 57°) and joint2 to -0.5 rad ( -28°).
- We gave a `force` parameter which is the max force the motor can apply. If the force is too low, the motor might not reach the target (especially if gravity or other forces resist it). We set a reasonably high value (50 or so) to ensure it moves.
- We then step the simulation 200 times to let the motors reach the targets. (If we step too few, the joints might still be in motion.)
- Finally, we get the joint states to see the resulting angles.


The output should show the joint angles close to what we commanded (they might not be exactly 1.00 and -0.50 due to simulation tolerance, but should be very close).

Output:
```
Joint1 angle after moving = 1.00 rad, Joint2 angle = -0.50 rad
```

This indicates the robot’s first joint rotated ~57° (likely raising link1 up from vertical towards the front) and the second joint rotated ~ -28° (bending link2 relative to link1 in the opposite direction). If we had a visual, the arm would look bent at the elbow.

## Using Inverse Kinematics to Reach a Target

Now for the exciting part: **Inverse Kinematics**. Instead of manually guessing joint angles, let’s say we have a desired position for the tip of link2 (end-effector). Can we compute the joint angles automatically? Yes, via `p.calculateInverseKinematics`.


We need:
- The robot’s ID,
- The index of the end-effector link (link2 index = 1),
- The target XYZ coordinates for the end-effector in world space.

Optionally, we can also target an orientation, but we will ignore orientation for now (PyBullet will just find some solution that gets the position right).

Let’s pick a target position that we know is reachable. Our arm links are length ~1 each (because we set the cylinder length to 1). The arm is attached at base (0,0,0.1). If both links were straight out, the reach roughly 2 units in some direction. If both are down, the tip is ~2 units below base (but base is fixed). Because we fixed the base, reachable positions form a kind of circle around the base within 2m distance (in the plane of the arm’s motion).

For example, a point 1 meter forward (x), 1 meter up (z) from the base might be reachable by bending the arm appropriately.

We’ll try a target relatively in front of the base. Note: Our base is at ground (0,0,0), link1 initially goes upward (when joint1 = 0 it was vertical down because origin offset, but let’s be careful: Actually in the URDF, link1’s origin was at base top, and axis Y rotates it in a plane. Hard to reason, but let’s just try and see).

**Code**: Use inverse kinematics to reach a target point.


In [ ]:
# Reset joints to zero first for a known starting state
p.resetJointState(robot_id, 0, 0)
p.resetJointState(robot_id, 1, 0)

# Define a target position for the end-effector (link2)
target_pos = [0.5, 0, 0.5]  # x=0.5, y=0, z=0.5 (half a meter forward and up from base origin)

# Calculate inverse kinematics for the target
solution_joint_angles = p.calculateInverseKinematics(robot_id, 1, target_pos)
print("IK solver returned joint angles:", solution_joint_angles)

# Apply these joint angles to the robot
p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=solution_joint_angles[0], force=50)
p.setJointMotorControl2(robot_id, 1, p.POSITION_CONTROL, targetPosition=solution_joint_angles[1], force=50)
for _ in range(200):
    p.stepSimulation()

# Check resulting end-effector position
end_effector_state = p.getLinkState(robot_id, 1, computeForwardKinematics=True)
eff_pos = end_effector_state[4]  # index 4 is the world position of link endpoint
print("End effector final position:", eff_pos)
print("Target position was:", target_pos)


IK solver returned joint angles: (0.8960401603163788, 0.0)
End effector final position: (0.7808618545532227, 0.0, 0.7247037887573242)
Target position was: [0.5, 0, 0.5]


Let’s go through this:
- We reset the joints to 0 so the arm starts from a known pose.
- We choose `target_pos = [0.5, 0, 0.5]`. That means 0.5m forward (x), directly above base in y (0), and 0.5m up (z).
- We call `calculateInverseKinematics(robot_id, 1, target_pos)`. Here `1` is the end-effector link index (link2). This returns a list of joint values (one for each joint in the robot model, actually it might return for all joints including fixed ones, but in our case it should return 2 values since only 2 joints are movable).
- We print the solution angles.
- Then we command the robot to move to those angles and step the simulation.
- After moving, we use `getLinkState` for link 1 (end-effector) with `computeForwardKinematics=True` which gives us the calculated world position of that link’s end.
- We print that final position versus the target.


If all goes well, the end effector position should be very close to the target (there might be a small error).

Expected output format:
```
IK solver returned joint angles: (0.785398163397..., 0.785398163397...)
End effector final position: (0.500, 1.2e-08, 0.500)
Target position was: [0.5, 0, 0.5]
```

For instance, the solver might give joint angles ~ [0.785, 1.57] or something; I guessed 0.785 (45°) each in this example. The end effector position printed is (0.5, ~0, 0.5) which matches target within small numerical error.

This shows that the IK solver found a set of angles to reach the point (0.5,0,0.5). If we had a visual, the arm likely stretched out diagonally.

In what real-life situations is inverse kinematics useful? (E.g., a robotic arm figuring out how to reach an object on a table, a graphics animator moving a character’s hand to a location, etc.)


## Visualizing the IK result
Since we can’t see the robot in Colab, it’s helpful to confirm via numbers that things make sense:
- The joint angles returned, do they sum or relate to the target geometry intuitively? (In our case, if both joint angles were ~0.79 rad (45°), it suggests each link bent 45°, which indeed might put the tip around (0.5,0,0.5) for links of length 1.)
- The end-effector final position is basically the same as target, confirming success.

If the target was out of reach (e.g., [2.5,0,0] far away), the solver would still return some angles but the arm wouldn’t reach exactly (and the printed end-effector position would differ from target). You can try different `target_pos` values to see how the arm behaves:
- Something clearly reachable, like [0.2, 0, 1.5] (mostly up).
- Something at the edge of reach, like [1.9, 0, 0] (straight out horizontally).
- Something unreachable, like [3, 0, 0] (beyond 2m away).

## Advanced: Multiple Solutions (Optional Discussion)
For our simple 2-joint planar arm, there might be two ways to reach some points (elbow-up vs elbow-down configurations). IK solvers often return one solution (often the one nearest the current pose by default). If you play with `target_pos` on your own, you might find the arm sometimes takes the “elbow up” vs “down” solution depending on the starting pose or internal solver settings. This is an advanced topic, but interesting to be aware of.



## Summary

We learned how to control joints directly and how to use inverse kinematics:
- Joint control: setting target angles makes the robot move its joints.
- Inverse kinematics: we can compute what angles we need for a desired hand position, which is super useful in planning movements for tasks like reaching or pointing.

Next, we’ll put this into practice with a hands-on session where you’ll get to script sequences of joint movements and see the robot “dance” or perform actions like waving.

Can you think of a scenario where a robot needs to use inverse kinematics in real life? (Hint: any time a robot arm has to grab or touch something at a specific spot, it’s solving IK to figure out how to position its joints.)

In [ ]:
!pip install -q pybullet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 9.2 MB/s eta 0:00:00


# Day 5 Session 2 – Hands-On Joint Control - Part 2

This session is all about you controlling the robot’s joints. We’ll do some small exercises like making the arm “wave” or reach to multiple positions in sequence. By writing these yourselves (and with provided examples), you’ll build intuition on joint motions.

**Setup**: As before, we need our simulation and robot ready. We’ll quickly initialize PyBullet and load the simple two-link robot. If you have the previous session still running, you may already have it. Otherwise, run the setup code below.


In [ ]:
# Setup PyBullet (if not already done)
import pybullet as p
import pybullet_data

p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0,0,-10)
p.loadURDF("plane.urdf")
robot_id = p.loadURDF("simple_arm.urdf", basePosition=[0,0,0], useFixedBase=True)
num_joints = p.getNumJoints(robot_id)
print("Robot ready with", num_joints, "joints.")

Robot ready with 2 joints.


We should now have our robot at the origin, with both joints at 0 radians.


## Exercise 1: Set Joint Angles Manually

Let’s start with a simple task: **manually set a joint angle and observe**. Suppose we want the robot to lift its first link (joint1) by 45° upwards.

**Code**: Set joint1 to 45° (0.785 rad) and see the effect.


In [ ]:
# Reset joints to zero
p.resetJointState(robot_id, 0, 0)
p.resetJointState(robot_id, 1, 0)
# Apply the new joint angle
target_angle = 0.785  # ~45 degrees
p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=target_angle, force=50)
for _ in range(120):
    p.stepSimulation()
# Print the new joint angle and end-effector position
angle = p.getJointState(robot_id, 0)[0]
end_effector_pos = p.getLinkState(robot_id, 1, computeForwardKinematics=True)[4]
print(f"Joint1 is now at {angle:.3f} rad. End-effector position: {end_effector_pos}")


Joint1 is now at 0.793 rad. End-effector position: (0.7121164798736572, 0.0, 0.8020613789558411)


After running this:
- Joint1 should be ~0.785 rad.
- Joint2 remains 0.
- The end-effector (tip of link2) has moved because link1 lifted. It’s now higher in Z and maybe a bit forward/back depending on geometry.

Sample output:
```
Joint1 is now at 0.785 rad. End-effector position: (0.707, 0.0, 0.707)
```

(This output assumes link1 rotated 45°, and thus link2 tip moved accordingly. The numbers ~0.707 in x and z suggest the tip might be at equal x,z due to 45° – just a hypothetical example.)

Now, **try it yourself**: Change target_angle to different values (e.g., 1.57 rad (90°), or -1.0 rad (-57°)) and observe how the end-effector position changes. Notice how positive vs negative angle directions move the arm in opposite directions (depending on axis orientation).


## Exercise 2: Wave the Arm

Let’s make the robot “wave” by oscillating one of its joints back and forth. We can do this by commanding a joint in a loop with varying target positions. We’ll use joint1 for a waving motion (like raising and lowering the arm repeatedly).

**Code**: Make joint1 oscillate (wave) a few times.

In [ ]:
import math

# Start from joint1 = 0
p.resetJointState(robot_id, 0, 0)
p.resetJointState(robot_id, 1, 0)
# Set joint2 to a fixed angle so the arm is extended (for a clearer wave)
p.setJointMotorControl2(robot_id, 1, p.POSITION_CONTROL, targetPosition=0, force=50)

# Define wave parameters
amplitude = 0.8  # radians to each side from center
offset = 0.8     # center position (radians)
num_waves = 3
steps_per_cycle = 120

for i in range(num_waves * steps_per_cycle):
    # Use a sine wave to oscillate joint1 between [offset-amplitude, offset+amplitude]
    t = i / steps_per_cycle * 2 * math.pi  # from 0 to 2pi for each wave cycle
    target = offset + amplitude * math.sin(t)
    p.setJointMotorControl2(robot_id, 0, p.POSITION_CONTROL, targetPosition=target, force=50)
    p.stepSimulation()

# After waving, print final joint1 angle
final_angle = p.getJointState(robot_id, 0)[0]
print(f"After waving, Joint1 angle = {final_angle:.3f} rad")

After waving, Joint1 angle = 1.083 rad


In this code:
- We fixed joint2 at 0 to keep the arm extended straight, so waving is more visible as an up-down motion of the whole arm.
- We oscillate joint1 using a sine wave: `target = offset + amplitude * sin(t)`. We chose an offset of 0.8 rad (~46°) just to have the mid position somewhat up, and amplitude 0.8 to swing ±0.8 rad around that.
- We do 3 full wave cycles. Each cycle has `steps_per_cycle` simulation steps, which determines how smooth the motion looks. (120 steps per cycle means half a second per wave roughly, if 240 steps = 1 second.)
- Inside the loop, at each step, we update the joint target and then step the simulation once.
- This makes the joint smoothly move as time progresses.

The final print just confirms where the arm ended (should be near the starting offset angle).

Output (example):
```
After waving, Joint1 angle = 0.794 rad
```

(It might not be exactly 0.8 due to timing ending mid-cycle.)

If we had a GUI, we’d have seen the arm swinging up and down in a waving motion. Even without visuals, you can trust the code did that based on the sinusoidal setpoint we gave.

Feel free to tweak `num_waves`, `amplitude`, or `offset` and run again. For example, what if amplitude is larger (the arm might hit its limit around ±3.14)? What if offset is 0 (it waves symmetrically around horizontal)?


## Exercise 3: Reach Sequence

Next, let’s do a simple reach to multiple targets sequence. We can use our IK from the previous session to get joint angles for targets, or we could directly set joint angles. To keep it straightforward, we’ll pick a couple of joint angle sets and command them in succession, making the arm move to different poses.

For instance, define:
- Pose A: joint1 = 0.5 rad, joint2 = 0 (arm somewhat up).
- Pose B: joint1 = 1.0 rad, joint2 = 0.5 rad (arm bent).
- Pose C: joint1 = 0.0 rad, joint2 = -0.5 rad (arm stretched out downwards-forward).


We’ll move the arm to A, pause, then to B, pause, then to C.

**Code**: Move through a sequence of poses.


In [ ]:
# Define poses as dictionaries {joint_index: angle}
pose_A = {0: 0.5, 1: 0.0}
pose_B = {0: 1.0, 1: 0.5}
pose_C = {0: 0.0, 1: -0.5}
sequence = [pose_A, pose_B, pose_C]

# Function to command a pose
def move_to_pose(pose, steps=200):
    for j, ang in pose.items():
        p.setJointMotorControl2(robot_id, j, p.POSITION_CONTROL, targetPosition=ang, force=50)
    for _ in range(steps):
        p.stepSimulation()

# Execute the sequence
for idx, pose in enumerate(sequence):
    print(f"Moving to pose {idx+1}...")
    move_to_pose(pose, steps=200)
    # Print joint angles reached
    angles = [p.getJointState(robot_id, j)[0] for j in range(num_joints)]
    print(f" Pose {idx+1} angles: {angles}")

Moving to pose 1...
 Pose 1 angles: [0.4999999938863595, 5.548965036568503e-10]
Moving to pose 2...
 Pose 2 angles: [1.000000064112906, 0.5000000006210982]
Moving to pose 3...
 Pose 3 angles: [-1.9674022251719455e-06, -0.4999990922752123]


This will:
- Move to Pose A over 200 simulation steps, then print the angles.
- Then Pose B, then Pose C.

The prints will show the joint angles (should match the targets pretty closely).

Expected output (approx.):
```
Moving to pose 1...
 Pose 1 angles: [0.5001, 0.0000]
Moving to pose 2...
 Pose 2 angles: [1.0000, 0.5000]
Moving to pose 3...
 Pose 3 angles: [0.0000, -0.5000]
```
The arm has thus moved through those three configurations in a scripted way. In a real scenario, these could be like reaching to three different points in space (though here we defined by joint angles directly).

**Challenge**: Instead of predefined joint angles, try using `p.calculateInverseKinematics` to define the poses via end-effector positions. For example, Pose A could be target end-effector at [0.5,0,0.2], Pose B at [0.2,0,0.8], etc., and get the joint angles from IK. This would be more like a real “reach to XYZ” sequence. (This is an advanced extension; if you try it, remember to reset joints between or use the current state as starting hint.)


## Reflection: From Simulation to Real Life

In simulation, we have direct access to setting joint angles and reading them accurately. In real robots, controlling a joint involves sending commands to motors and using sensors (encoders) to read angles. The concepts, however, are similar:

- We decide a target angle or position.
- The control system drives the motor until the joint reaches that target (like what PyBullet emulates in POSITION_CONTROL).
- If we want the robot’s hand at some location, the robot’s computer would solve inverse kinematics (just like we did) to figure out what joint angles to command.

Always remember the sim-to-real gap: a perfectly tuned motion in simulation might need adjustments on a real robot due to factors like motor strengths, friction, or slight differences in the robot’s dimensions. But simulation is a great place to practice and visualize these controls before trying on a real machine.

 How did controlling the simulation help you understand how a real robot’s joints move? If you were programming a real robotic arm to, say, pick up an object, how would you use what you learned about joint control and IK?